# Greenplum Database  Concepts Explained - Part 1
## 1. System Setup
### 1.1 Initialize database connection and setup global variable values

In [1]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

'Connected: gpadmin@gpadmin'

In [2]:
%%sql $DB_USER@$DB_SERVER
SHOW gp_autostats_mode;
ALTER DATABASE gpadmin SET gp_autostats_mode TO 'NONE';
SHOW gp_autostats_mode;

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
NONE


In [3]:
%%sql $DB_USER@$DB_SERVER
SELECT version();

1 rows affected.


version
"PostgreSQL 8.3.23 (Greenplum Database 5.20.1 build commit:03ff833f877a23469ca41aab0b2dfc58c48520ad) on x86_64-pc-linux-gnu, compiled by GCC gcc (GCC) 6.2.0, 64-bit compiled on Jun 28 2019 08:56:11"


## 2. The Amazon Customer Reviews Dataset

Over 130+ million customer reviews are available to researchers as part of this release. The data is available in TSV files in the `amazon-reviews-pds` S3 bucket in AWS US East Region. Each line in the data files corresponds to an individual review (tab delimited, with no quote and escape characters). Samples of the data are available in English and French; more details on the information in each column can be found [here](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt).

If you use the AWS Command Line Interface, you can list data in the bucket with the `aws s3 ls` command:

`aws s3 ls s3://amazon-reviews-pds/tsv/`

To download data using the AWS Command Line Interface, you can use the `aws s3 cp` command. For instance, the following command will copy the file named `amazon_reviews_us_Camera_v1_00.tsv.gz` to your local directory:

`aws s3 cp s3://amazon-reviews-pds/tsv/<S3 File> <Local File>`

### 2.1 Prepare AWS System and setup awscli library via pip

In [4]:
shfilecode = !pygmentize -f html -O full,style=friendly -l shell script/1-1-system-prepare.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 cd $HOME 

curl -O https://bootstrap.pypa.io/get-pip.py
python $HOME /get-pip.py --force-reinstall
rm $HOME /get-pip.py

mv $HOME /.bash_profile $HOME /.bash_profile.old

cat >> $HOME /.bash_profile << 'EOF' 
 # .bash_profile 

 # Get the aliases and functions 
 if [ -f ~/.bashrc ]; then 
 . ~/.bashrc 
 fi 

 # User specific environment and startup programs 

 export PATH=$HOME/.local/bin:$HOME/bin:$PATH 

 EOF 

chmod 644 $HOME /.bash_profile
 source $HOME /.bash_profile

pip install --upgrade --force-reinstall pip
pip install --upgrade --force-reinstall boto3
pip install --upgrade --force-reinstall awscli

 echo "***********************" 
 echo "* Get AWS CLI version *" 
 echo "***********************" 
aws --version

In [5]:
!ssh-keygen -R $DB_SERVER
!ssh-keyscan $DB_SERVER >> ~/.ssh/known_hosts
!scp -i ~/.ssh/aws-gp.pem script/1-1-system-prepare.sh $DB_USER@$DB_SERVER:system-prepare.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./system-prepare.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./system-prepare.sh'

Host ec2-35-177-141-170.eu-west-2.compute.amazonaws.com not found in /root/.ssh/known_hosts
# ec2-35-177-141-170.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-35-177-141-170.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
# ec2-35-177-141-170.eu-west-2.compute.amazonaws.com:22 SSH-2.0-OpenSSH_7.4
1-1-system-prepare.sh                         100%  712    76.5KB/s   00:00    
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1733k  100 1733k    0     0  12.2M      0 --:--:-- --:--:-- --:--:-- 12.2M
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  Found existi

  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Created wheel for PyYAML: filename=PyYAML-5.1.2-cp27-cp27mu-linux_x86_64.whl size=44890 sha256=357577573bd1ec2b2a9d82e03bdfefad2c633bec6ae71428ae3569f2169e75c0
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built PyYAML
  Found existing installation: docutils 0.15.2
    Uninstalling docutils-0.15.2:
      Successfully uninstalled docutils-0.15.2
  Found existing installation: PyYAML 3.10
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
***********************
* Get AWS CLI version *
***********************
aws-cli/1.15.30 Python/2.7.5 Linux/3.10.0-957.21.3.el7.x86_64 botocore/1.10.30


### 2.2 Provide AWS Access Key ID & Secret Access Key

In [6]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-2-aws-configure.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 if [ -d $HOME /.aws ] ; then rm -rf $HOME /.aws ; fi 

mkdir $HOME /.aws

 echo "[default] 
 aws_access_key_id=" $1 " 
 aws_secret_access_key=" $2 "" > $HOME /.aws/credentials

 echo "[default] 
 region=eu-west-2 
 output=json" > $HOME /.aws/config

 #test aws config 
 if [ " $( /usr/local/bin/aws s3 ls s3://amazon-reviews-pds/tsv/ | awk '{print $4}' | awk 'END {print NR}' ) " -eq "0" ] ; 
 then echo "Error on AWS S3 Configuration setup" ; 
 else echo "AWS S3 Configuration setup correctly" ; 
 fi

In [7]:
import getpass

!scp -i ~/.ssh/aws-gp.pem script/1-2-aws-configure.sh $DB_USER@$DB_SERVER:aws-configure.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-configure.sh'

cmd = 'sudo ./aws-configure.sh ' 
cmd = cmd + getpass.getpass("AWS Access Key ID [None]:") 
cmd = cmd + ' ' + getpass.getpass("AWS Secret Access Key [None]:")

!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

1-2-aws-configure.sh                          100%  484    55.9KB/s   00:00    
AWS Access Key ID [None]:········
AWS Secret Access Key [None]:········
AWS S3 Configuration setup correctly


### 2.3 Copy source files from AWS S3
For our demo, we choose to download the available files into the `/home/gpadmin/data/` folder, using the `aws s3 cp <S3 File> <Local File>` command described before, as follows:

In [8]:
shfilecode = !pygmentize -f html -O full,style=friendly -l bash script/1-3-aws-s3-copy.sh
display_html('\n'.join(shfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 #!/bin/bash 

 export S3_DATA_DIR = /var/tmp_s3_data

 if [ -d $S3_DATA_DIR ] ; then rm -rf $S3_DATA_DIR ; fi 

mkdir $S3_DATA_DIR 
 cd $S3_DATA_DIR 

ls -laF $S3_DATA_DIR 

/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz amazon_reviews_us_Books_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz amazon_reviews_us_Books_v1_01.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz amazon_reviews_us_Wireless_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz amazon_reviews_us_Music_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz amazon_reviews_us_PC_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz amazon_reviews_us_Video_DVD_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_multilingual_US_v1_00.tsv.gz amazon_reviews_multilingual_US_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz amazon_reviews_us_Books_v1_02.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz amazon_reviews_us_Home_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz amazon_reviews_us_Kitchen_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz amazon_reviews_us_Beauty_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz amazon_reviews_us_Sports_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz amazon_reviews_us_Toys_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz amazon_reviews_us_Electronics_v1_00.tsv.gz
/usr/local/bin/aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz amazon_reviews_us_Apparel_v1_00.tsv.gz

In [9]:
!scp -i ~/.ssh/aws-gp.pem script/1-3-aws-s3-copy.sh $DB_USER@$DB_SERVER:aws-s3-copy.sh
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'chmod +x ./aws-s3-copy.sh'
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'sudo ./aws-s3-copy.sh'

1-3-aws-s3-copy.sh                            100% 2590   441.1KB/s   00:00    
total 4
drwxr-xr-x   2 root root    6 Sep 27 10:59 ./
drwxr-xr-x. 21 root root 4096 Sep 27 10:59 ../
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz to ./amazon_reviews_us_Books_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz to ./amazon_reviews_us_Books_v1_01.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz to ./amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz to ./amazon_reviews_us_Wireless_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz to ./amazon_reviews_us_Music_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz to ./amazon_reviews_us_PC_v1_00.tsv.gz
download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz to .

## 3. Data Loading
### 3.1. Create the Schema (optional) and the Database Table to hold the dataset, as shown below:

In [10]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-1-create-db-schema-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP SCHEMA IF EXISTS demo CASCADE ; 

 CREATE SCHEMA demo ; 

 DROP TABLE IF EXISTS demo . amzn_reviews ; 

 CREATE TABLE demo . amzn_reviews ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id );

In [11]:
query = !cat script/2-1-create-db-schema-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
Done.
Done.


[]

In [12]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/2-2-count-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [13]:
query = !cat script/2-2-count-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


count
0


### 3.2. Load the Input Dataset using the gpload Utility
**gpload** is a data loading utility that acts as an interface to the Greenplum Database external table parallel loading feature. Using a load specification defined in a YAML formatted control file, gpload executes a load by invoking the Greenplum Database parallel file server (**gpfdist**), creating an external table definition based on the source data defined, and executing an *INSERT*, *UPDATE* or *MERGE* operation to load the source data into the target table in the database.

You can declare more than one file as input/source as long as the data is of the same format in all files specified. Additionally, if the files are compressed using **gzip** or **bzip2** (have a .gz or .bz2 file extension), the files will be uncompressed automatically (provided that gunzip or bunzip2 is in your path). You can also declare options such as the schema of the source data files, perform basic transformations, define custom delimiter and/or escape character(s), and many more. For the full list of available options, check the GPLoad Utility Reference available on [Pivotal Greenplum Database Documentation](https://gpdb.docs.pivotal.io/latest) (*Pivotal Greenplum Documentation > Utility Guide > Management Utility Reference > gpload*).

The operation, including any SQL commands specified in the SQL collection of the YAML control file, are performed as a single transaction to prevent inconsistent data when performing multiple, simultaneous load operations on a target table.

For our demo, we have prepared the *gpload_amzn_reviews.yaml* YAML control file, as shown here:

In [14]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l yaml script/3-2-gpload-amzn-reviews.yaml
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 VERSION : 1.0.0.1 
 GPLOAD : 
 INPUT : 
 - SOURCE : 
 FILE : 
 - /var/tmp_s3_data/amazon_reviews_us*.tsv.gz 
 - FORMAT : text 
 - HEADER : true 
 - LOG_ERRORS : true 
 - MAX_LINE_LENGTH : 1000000 
 - ERROR_LIMIT : 50000 
 OUTPUT : 
 - TABLE : demo.amzn_reviews 
 - MODE : insert 
 PRELOAD : 
 - TRUNCATE : true 
 - REUSE_TABLES : true

#### 3.2.1. Delete error log information for existing tables in the current database.

In [15]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-1-delete-error-log-info.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT gp_truncate_error_log ( 'demo.amzn_reviews' );

In [16]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload_amzn_reviews.log ]; then rm ./gpload_amzn_reviews.log; fi'

query = !cat script/3-1-delete-error-log-info.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


gp_truncate_error_log
True


#### 3.2.2. Copy GPLoad YAML file across to the Database Server and execute

In [17]:
!scp -i ~/.ssh/aws-gp.pem script/3-2-gpload-amzn-reviews.yaml $DB_USER@$DB_SERVER:gpload_amzn_reviews.yaml

cmd = "gpload -d {0} -f ./gpload_amzn_reviews.yaml -l ./gpload_amzn_reviews.log 2>&1".format(DB_USER) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

3-2-gpload-amzn-reviews.yaml                  100%  356    76.9KB/s   00:00    
2019-09-26 15:07:06|INFO|gpload session started 2019-09-26 15:07:06
2019-09-26 15:07:06|INFO|no host supplied, defaulting to localhost
2019-09-26 15:07:06|INFO|started gpfdist -p 8000 -P 9000 -f "/var/tmp_s3_data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2019-09-26 15:07:06|INFO|did not find an external table to reuse. creating ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e
2019-09-26 15:13:50|WARN|3714 bad rows
2019-09-26 15:13:50|WARN|Please use following query to access the detailed error
2019-09-26 15:13:50|WARN|select * from gp_read_error_log('ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e') where cmdtime > to_timestamp('1569506826.77')
2019-09-26 15:13:50|INFO|running time: 403.87 seconds
2019-09-26 15:13:50|INFO|rows Inserted          = 103145273
2019-09-26 15:13:50|INFO|rows Updated           = 0
2019-09-26 15:13:50|INFO|data formatting errors = 3714
2019-09-26 15:13:50|INFO|gp

### 3.3. Check gpload execution

Check **gpload** execution output (shown above and also available on *./gpload_amzn_reviews.log*), confirm successful loading of the data and/or identify any message which require ones attention and/or actions:

#### 3.3.1. Check the data has been properly loaded, by confirming row count shown above:

In [18]:
sqlfilecode = !pygmentize -f html -O full,style=friendly -l postgres script/3-3-count-amzn-reviews.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT COUNT ( * ) FROM demo . amzn_reviews ;

In [19]:
query = !cat script/3-3-count-amzn-reviews.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


count
103145273


In [20]:
cmd = 'cat /home/gpadmin/gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|";OFS=" "} {print $3}'"'"'\
    | awk '"'"'{print $1, "COUNT(*)", $3, $4, $5, $6, $7, $8}'"'"''
query = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd
%sql $DB_USER@$DB_SERVER {''.join(query)}

1 rows affected.


count
3714


#### 3.3.3. Check a sample set of 10 rows from the data formatting errors, if such were identified by the gpload execution log:

In [21]:
cmd = 'cat /home/gpadmin/gpload_amzn_reviews.log\
    | grep -e '"'"'WARN|select'"'"'\
    | awk '"'"'BEGIN{FS="|"} {print $3, "LIMIT 10"}'"'"' ' 
query = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd
%sql $DB_USER@$DB_SERVER {''.join(query)}

10 rows affected.


cmdtime,relname,filename,linenum,bytenum,errmsg,rawdata,rawbytes
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/var/tmp_s3_data/amazon_reviews_us_Apparel_v1_00.tsv.gz],None,None,"missing data for column ""review_date""",US 372288 RI0EFHWZ9XAXO B00IR3XOUU 428316924 Bunnies By The Bay Baby-Boys Newborn Sweet Sailor Romper Apparel 5 0 0 N Y Five Stars thanx\ 2015-07-14,None
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/var/tmp_s3_data/amazon_reviews_us_Apparel_v1_00.tsv.gz],None,None,"missing data for column ""review_date""","US 23766352 R1TPKN3W4DUQAU B00H4MYVG6 294206631 MBJ Womens Active Soft Zip Up Fleece Hoodie Sweater Jacket Apparel 4 2 3 N Y Not too bad, fits small It came very fast! Faster than i expected! So that was awesome. However the color was not as bright blue as shown in the picture. It was more of a navy blue, so that was kind of disappointing. I ordered a size larger than normal and i'm glad that i did. I ordered a large and i usually get a medium or small and i'm glad that i got a large because even after i washed it the sleeves seemed a little small. I like the length though. this coat was VERY comfy too, and it has a nice hood for when you want to chill in the hood. The pockets are pretty nice and don't puff up or lose things. Over all its a good coat, i just wish the color was true :-\ 2014-03-07",None
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/var/tmp_s3_data/amazon_reviews_us_Apparel_v1_00.tsv.gz],None,None,"missing data for column ""review_date""","US 29732584 R3RDDEZ7H73K5B B00FHO70LG 856305243 Allegra K Women's Novelty Prints Pockets Front Lined Hoodie Apparel 2 0 1 N Y Fake Took forever to get it. Shipped from China. It's a fake. Not an Allegra K product. Contacted shipper to return but looks like it will cost me more in return shipping then what I paid for it. Huge bummer. I wanted this big time. Oh, I bought the brown one in the largest size available. It's CRAZY small. Child sized. :\ 2014-12-02",None
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/var/tmp_s3_data/amazon_reviews_us_Apparel_v1_00.tsv.gz],None,None,"missing data for column ""review_date""","US 18230669 R3U1DW0ZBUL9TB B00A7QUFSO 788432831 Hanes Men's Classics Power Slim Crew Neck T-Shirt (Pack of 2) Apparel 1 0 1 N Y NOT SIZE MEDIUM, MADE FOR TRIANGLE PEOPLE Holy crap who can wear these? I wear a universal medium like all my life and figured id would get these because they would be easier to tuck in as undershirts but I cant even wear them, im 120lbs and not very large and I cant even put on these tees, they taper down to a baby waist. I ended up giving them to my 105lb female friend in hopes that they would fit her, get one ir two sizes larger than you are unless you are shaped like a triangle, and if you are then good luck getting them on over your shoulders :\ 2015-06-26",None
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/var/tmp_s3_data/amazon_reviews_us_Beauty_v1_00.tsv.gz],None,None,"missing data for column ""review_date""",US 10565880 R1O675WJRYFKG3 B007X7IO4M 74481432 Avon Mineral Gems Glamorous Gold Shimmering Body Oil Spray Beauty 3 0 0 N Y I was really looking for the oil that rubs on like a lotion in the squeez tube Spray pump did'nt fit bottle! I was really looking for the oil that rubs on like a lotion in the squeez tube :-\ 2014-07-25,None
2019-09-26 15:07:06.930462+01:00,ext_gpload_reusable_ec53c9c8_e066_11e9_846a_0621ba98679e,gpfdist://mdw:8000//var/tmp_s3_data/amazon_reviews_us*.tsv.gz [/

### Continue to Part 2 of Greenplum Database Concepts Explained; [Basic Table Functions](AWS-GP-demo-2.ipynb).